# **Data Preprocessing**

This notebook handles data cleaning, feature engineering, and preparation for modeling based on insights from the exploratory data analysis. The preprocessing pipeline ensures no data leakage by performing train/test split before any statistical operations like outlier removal or scaling.

The cleaned dataset will be saved both locally and uploaded to S3 for use in the modeling phase.

In [40]:
import pandas as pd
import boto3
import io
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [25]:
# FETCH DATA FROM AWS

# Create the AWS S3 Client
s3_client = boto3.client('s3')
# Load the data in a dictionary 'response'
response = s3_client.get_object(Bucket = 'software-tools-ai', Key ='raw_data/listings.csv')
# Filter the content of the csvg
csv_content = response['Body'].read()

In [26]:
# Load fetched data into the dataset
df = pd.read_csv(io.BytesIO(csv_content), header = 0, sep = ',')
print(df.shape)
df.head(2)


(48895, 16)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355


### Outlier Handling Strategy

Based on EDA findings, price has significant outliers (up to $10,000). This preprocessing pipeline implements an **experimental baseline strategy**: removing listings above the 99th percentile ($799).

**This is one of several strategies to be tested during modeling:**
- Strategy 1 (this notebook): Remove outliers > 99th percentile
- Strategy 2 (modeling phase): Log transformation to compress extremes
- Strategy 3 (modeling phase): Keep all data, use robust models (Random Forest, XGBoost)

The final outlier handling approach will be selected based on model performance comparison tracked in MLflow.

In [27]:
# Data cleaning before split

# Remove invalid prices
print(f"Original dataset: {df.shape}")
df = df[df['price'] > 0]
print(f"After removing $0 prices: {df.shape}")

# Drop unnecessary columns
cols_to_drop = ['id', 'host_id', 'name', 'host_name', 'neighbourhood']
df = df.drop(columns=cols_to_drop)
print(f"After dropping ID columns: {df.shape}")
print(f"Remaining columns: {list(df.columns)}")

Original dataset: (48895, 16)
After removing $0 prices: (48884, 16)
After dropping ID columns: (48884, 11)
Remaining columns: ['neighbourhood_group', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']


### Train-Test-Splt

In [28]:
# Separate features and target
X = df.drop('price', axis=1)
y = df['price']

# Split 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Split ratio: {X_train.shape[0]/len(df)*100:.1f}% train, {X_test.shape[0]/len(df)*100:.1f}% test")

Training set: 39107 samples
Test set: 9777 samples
Split ratio: 80.0% train, 20.0% test


In [29]:
# Calculate outlier threshold on TRAIN only
outlier_threshold = y_train.quantile(0.99)
print(f"Outlier threshold (99th percentile from train): ${outlier_threshold:.2f}")

# Count outliers in train
n_outliers_train = (y_train > outlier_threshold).sum()
print(f"Outliers in training set: {n_outliers_train}")

# Remove outliers from TRAIN
mask = y_train <= outlier_threshold
X_train = X_train[mask]
y_train = y_train[mask]

print(f"Training set after outlier removal: {X_train.shape[0]} samples")
print(f"Test set (unchanged): {X_test.shape[0]} samples")

Outlier threshold (99th percentile from train): $798.76
Outliers in training set: 392
Training set after outlier removal: 38715 samples
Test set (unchanged): 9777 samples


In [30]:
# Check missing values in train
missing_df = pd.DataFrame({
    'column': X_train.columns,
    'missing_count': X_train.isnull().sum().values,
    'missing_pct': (X_train.isnull().sum() / len(X_train) * 100).round(2).values
})

missing_df = missing_df[missing_df['missing_count'] > 0].sort_values('missing_count', ascending=False)

print("Missing values in training set:")
missing_df

Missing values in training set:


,column,missing_count,missing_pct
6,last_review,7833,20.23
7,reviews_per_month,7833,20.23


In [31]:
# Handle missing values

# Impute reviews_per_month with 0 (listings without reviews)
X_train['reviews_per_month'] = X_train['reviews_per_month'].fillna(0)
X_test['reviews_per_month'] = X_test['reviews_per_month'].fillna(0)

# Drop last_review (not useful for prediction)
X_train = X_train.drop(columns=['last_review'])
X_test = X_test.drop(columns=['last_review'])

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"\nMissing values remaining in train: {X_train.isnull().sum().sum()}")
print(f"Missing values remaining in test: {X_test.isnull().sum().sum()}")

Training set shape: (38715, 9)
Test set shape: (9777, 9)

Missing values remaining in train: 0
Missing values remaining in test: 0


## Preprocessing Summary (So Far)

The dataset has been cleaned and split for modeling. Invalid prices ($0) were removed, reducing the dataset from 48,895 to 48,884 listings. Unnecessary identifier columns (id, host_id, name, host_name) and the neighbourhood column were dropped. Neighbourhood had 221 unique categories which would create high dimensionality with one-hot encoding, so neighbourhood_group (5 boroughs) was kept instead as it captures location patterns more efficiently.

After an 80/20 train/test split, outliers above the 99th percentile ($798.76) were removed from the training set only, eliminating 392 listings. This experimental baseline strategy will be compared against alternative approaches (log transformation, robust models) during modeling.

Missing values in review columns (~20%) were imputed with 0 for `reviews_per_month` (representing listings without reviews) and `last_review` was dropped as it doesn't provide useful predictive information.

**Current dataset:**
- Training: 38,715 samples, 9 features
- Test: 9,777 samples, 9 features
- No missing values remaining

Next step is feature engineering to create additional predictive signals from geographic coordinates.

## Feature Engineering

Based on EDA findings showing Manhattan as the most expensive area, a new feature was created: distance from each listing to Manhattan's center (Times Square coordinates: 40.7580, -73.9855).

In [32]:
# Manhattan center coordinates
manhattan_lat = 40.7580
manhattan_lon = -73.9855

# Calculate Euclidean distance for train
X_train['distance_to_manhattan'] = np.sqrt(
    (X_train['latitude'] - manhattan_lat)**2 + 
    (X_train['longitude'] - manhattan_lon)**2
)

# Calculate for test
X_test['distance_to_manhattan'] = np.sqrt(
    (X_test['latitude'] - manhattan_lat)**2 + 
    (X_test['longitude'] - manhattan_lon)**2
)

print(f"Distance range in train: {X_train['distance_to_manhattan'].min():.4f} to {X_train['distance_to_manhattan'].max():.4f}")

Distance range in train: 0.0007 to 0.3631


The distance is calculated using Euclidean distance in geographic coordinates (latitude/longitude degrees). While this doesn't account for Earth's curvature, it's sufficient for NYC's small geographic area. The resulting values are in degrees rather than kilometers, but this doesn't affect model performance since what matters is the relative ordering (closer vs farther from Manhattan).

Distance range: 0.0007 to 0.3631 degrees, which translates roughly to 80 meters (very close to Times Square) up to ~40km (outer boroughs like Staten Island).

In [33]:
categorical_cols = ['neighbourhood_group', 'room_type']

numeric_cols = ['latitude', 'longitude', 'minimum_nights', 
                'number_of_reviews', 'reviews_per_month', 
                'calculated_host_listings_count', 'availability_365',
                'distance_to_manhattan']

print(f"Categorical: {categorical_cols}")
print(f"Numeric: {numeric_cols}")
print(f"Total: {len(categorical_cols) + len(numeric_cols)} features")

Categorical: ['neighbourhood_group', 'room_type']
Numeric: ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'distance_to_manhattan']
Total: 10 features


In [38]:
preprocessor = ColumnTransformer(transformers=[
    ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_cols),
    ('stdscaler', StandardScaler(), numeric_cols)
])

preprocessor.fit(X_train)
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print(f'X_train_processed shape: {X_train_processed.shape} \nX_test_processed shape: {X_test_processed.shape} \n')
print(f'X_train_processed shape: {type(X_train_processed)} \nX_test_processed shape: {type(X_test_processed)} \n')

X_train_processed shape: (38715, 14) 
X_test_processed shape: (9777, 14) 

X_train_processed shape: <class 'numpy.ndarray'> 
X_test_processed shape: <class 'numpy.ndarray'> 



In [ ]:
# Encoding and Scaling

# Define column types
categorical_cols = ['neighbourhood_group', 'room_type']
numeric_cols = ['latitude', 'longitude', 'minimum_nights', 
                'number_of_reviews', 'reviews_per_month', 
                'calculated_host_listings_count', 'availability_365',
                'distance_to_manhattan']

# Create preprocessing pipeline
# - OneHotEncoder: converts categorical variables to binary columns (drop first to avoid multicollinearity)
# - StandardScaler: normalizes numeric features to mean=0, std=1
preprocessor = ColumnTransformer(transformers=[
    ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_cols),
    ('stdscaler', StandardScaler(), numeric_cols)
])

# Fit preprocessor on training data (learn statistics)
preprocessor.fit(X_train)

# Transform both train and test using the same statistics
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Verify shapes and data type
print(f'X_train_processed shape: {X_train_processed.shape}')
print(f'X_test_processed shape: {X_test_processed.shape}')
print(f'\nX_train_processed type: {type(X_train_processed)}')
print(f'X_test_processed type: {type(X_test_processed)}')

### Saving Preprocessed Dataset - S3 and Local

In [44]:
# Save processed datasets

# Convert numpy arrays back to DataFrames and combine with target variable
# Note: X_processed has no column names (it's a numpy array after ColumnTransformer)
# Create generic feature names
feature_names = [f'feature_{i}' for i in range(X_train_processed.shape[1])]

# Combine X and y for train
train_processed = pd.DataFrame(X_train_processed, columns=feature_names, index=X_train.index)
train_processed['price'] = y_train

# Combine X and y for test
test_processed = pd.DataFrame(X_test_processed, columns=feature_names, index=X_test.index)
test_processed['price'] = y_test

# Create output directory
output_dir = '../data/processed'
os.makedirs(output_dir, exist_ok=True)

# Save locally
train_path = os.path.join(output_dir, 'train_processed.csv')
test_path = os.path.join(output_dir, 'test_processed.csv')
print(train_path)
train_processed.to_csv(train_path, index=False)
test_processed.to_csv(test_path, index=False)
print(f"✓ Saved locally: {train_path}")
print(f"✓ Saved locally: {test_path}")

# Upload to S3
for filename, df in [('train_processed.csv', train_processed), ('test_processed.csv', test_processed)]:
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)
    
    s3_client.put_object(
        Bucket='software-tools-ai',
        Key=f'processed_data/{filename}',
        Body=csv_buffer.getvalue()
    )
    print(f"✓ Uploaded to S3: s3://software-tools-ai/processed_data/{filename}")

print(f"\nFinal datasets:")
print(f"Train: {train_processed.shape}")
print(f"Test: {test_processed.shape}")

../data/processed\train_processed.csv
✓ Saved locally: ../data/processed\train_processed.csv
✓ Saved locally: ../data/processed\test_processed.csv
✓ Uploaded to S3: s3://software-tools-ai/processed_data/train_processed.csv
✓ Uploaded to S3: s3://software-tools-ai/processed_data/test_processed.csv

Final datasets:
Train: (38715, 15)
Test: (9777, 15)


## Preprocessing Complete

The dataset has been successfully preprocessed and is ready for modeling. The pipeline applied:

**Data cleaning:**
- Removed 11 listings with price = $0
- Dropped identifier columns and neighbourhood (high cardinality)

**Train/test split:**
- 80/20 split (38,715 train / 9,777 test samples)
- Outliers removed from train only (392 listings above $798.76)

**Feature engineering:**
- Created distance_to_manhattan feature from geographic coordinates
- Imputed missing review values with 0

**Encoding and scaling:**
- One-hot encoded categorical variables (neighbourhood_group, room_type) with drop_first
- StandardScaler applied to numeric features
- All transformations fit on train data only to prevent leakage

**Final datasets:**
- Training: 38,715 samples × 15 features (14 predictors + price)
- Test: 9,777 samples × 15 features
- Saved locally: `data/processed/train_processed.csv`, `test_processed.csv`
- Uploaded to S3: `s3://software-tools-ai/processed_data/`

Next step: Model development with MLflow experiment tracking.